In [1]:
from splitting.disaggregate import split_datapoint
from splitting.disaggregate import split_dataframe

import numpy as np
from splitting.models import RateMultiplicativeModel
from splitting.models import LMO_model
from splitting.models import LogOdds_model
import pandas as pd
from test_dataset import data_df,population_sizes,baseline_patterns,groups_to_split_into


In [2]:
display(data_df,population_sizes,baseline_patterns)

,pop_id,baseline_id,0,1,2,3,obs,obs_se
0,0,0,1,1,0,0,7.0,1.0
1,0,0,0,0,1,1,20.0,2.0
2,1,0,1,1,1,0,10.0,3.0
3,1,0,0,0,0,1,11.0,1.0
4,2,1,1,1,1,1,10.0,1.5
5,3,1,1,0,0,0,0.8,0.1
6,3,1,0,1,0,0,8.0,1.0
7,3,1,0,0,1,1,22.0,3.0


,0,1,2,3
pop_id,,,,
0,10,10,20,20
1,10,20,20,13
2,20,20,10,10
3,5,40,30,20


,0,1,2,3
baseline_id,,,,
0,0.4,0.20,0.3,0.75
1,0.1,0.15,0.2,0.70


In [3]:
split_dataframe(
    groups_to_split_into,
    data_df,
    population_sizes,
    baseline_patterns
)

,0,1,2,3,total
pop_id,,,,,
0,4.587886,2.412114,5.485838,14.514162,27.0
1,2.976175,2.742213,4.281612,11.000000,21.0
2,1.195737,1.834592,1.251663,5.718007,10.0
3,0.800000,8.000000,7.121403,14.878597,30.8


In [4]:
test_df=data_df.copy()
id_map={
    0:'a',
    1:'b',
    2:'c',
    3:'d'
}
test_df['pop_id']=test_df['pop_id'].map(id_map)
test_sizes=population_sizes.copy()
test_sizes.index=list(map(lambda x:id_map[x],test_sizes.index))

newgroups=['a','b','c','d']
test_df.columns=['pop_id', 'baseline_id', 'a','b','c','d', 'obs', 'obs_se']
test_sizes.columns=newgroups
new_baseline=baseline_patterns.copy()
new_baseline.columns=newgroups

In [9]:
split_dataframe(
    newgroups,
    test_df,
    test_sizes,
    new_baseline,
    use_se=False
)

,a,b,c,d,total
pop_id,,,,,
a,4.587886,2.412114,5.485838,14.514162,27.0
b,2.976175,2.742213,4.281612,11.000000,21.0
c,1.195737,1.834592,1.251663,5.718007,10.0
d,0.800000,8.000000,7.121403,14.878597,30.8


In [ ]:
splitting_df=test_df.copy()
model=LogOdds_model()
def split_row(x):
    try:
        return split_datapoint(
                x['obs'],
                test_sizes.loc[x.name]*x[newgroups],
                new_baseline.loc[x['baseline_id']],
                model=model
            )
    except:
        print(population_sizes.loc[x.name])
result=(
    splitting_df
    .set_index('pop_id')
    .apply(
        split_row,
        axis=1)
    .reset_index()
    .groupby('pop_id')
    .sum()
)
result['total']=result.sum(axis=1)
result


In [83]:
splitting_df=test_df.copy()
model=LogOdds_model()
def split_row(x):
    try:
        raw_split_result=split_datapoint(
                x['obs'],
                test_sizes.loc[x.name]*x[newgroups],
                new_baseline.loc[x['baseline_id']],
                model=model,
                measured_count_se=x['obs_se']
            )
        return pd.Series(
            [
                (estimate,se) for estimate,se in zip(raw_split_result[0],raw_split_result[1])],
            index=newgroups)
    except:
        print(population_sizes.loc[x.name])
result_raw=(
    splitting_df
    .set_index('pop_id')
    .apply(
        split_row,
        axis=1)
)
point_estimates=result.applymap(lambda x:x[0])
standard_errors=result.applymap(lambda x:x[1])
full_result=pd.concat([ point_estimates, standard_errors ], keys=['estimate','se'],axis=1).groupby(level=0).sum()


In [82]:
full_result

estimate                                       se                      \
               a         b         c          d         a         b         c   
pop_id                                                                          
a       4.587886  2.412114  5.485838  14.514162  0.575665  0.424335  1.000000   
b       2.976175  2.742213  4.281612  11.000000  0.801781  0.907569  1.290650   
c       1.195737  1.834592  1.251663   5.718007  0.266241  0.394610  0.259314   
d       0.800000  8.000000  7.121403  14.878597  0.100000  1.000000  1.763117   

                  
               d  
pop_id            
a       1.000000  
b       1.000000  
c       0.579834  
d       1.236883

In [43]:

split_row(splitting_df.set_index('pop_id').iloc[0])

a     (4.587886199200495, 0.5756649908457978)
b    (2.412113800799504, 0.42433500915420225)
c                                  (0.0, 0.0)
d                                  (0.0, 0.0)
dtype: object

In [32]:
a=split_row(splitting_df.set_index('pop_id').iloc[0])[:2]

In [51]:
df = pd.DataFrame({'a':[1,2], 'b':[(1,2), (3,4)]})
df

,a,b
0,1,"(1, 2)"
1,2,"(3, 4)"


In [52]:
df['b'].tolist()

[(1, 2), (3, 4)]

In [53]:
pd.DataFrame(df['b'].tolist(), index=df.index)

,0,1
0,1,2
1,3,4


In [54]:
df[['b1', 'b2']] = pd.DataFrame(df['b'].tolist(), index=df.index)

In [81]:
data_df

,pop_id,baseline_id,0,1,2,3,obs,obs_se
0,0,0,1,1,0,0,7.0,1.0
1,0,0,0,0,1,1,20.0,2.0
2,1,0,1,1,1,0,10.0,3.0
3,1,0,0,0,0,1,11.0,1.0
4,2,1,1,1,1,1,10.0,1.5
5,3,1,1,0,0,0,0.8,0.1
6,3,1,0,1,0,0,8.0,1.0
7,3,1,0,0,1,1,22.0,3.0
